# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [12]:
from sklearn.datasets import  fetch_california_housing
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [13]:
df = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [14]:
df = df.dropna()

df_cabin = df['Cabin']

def clean_cabin(x):
    return x[0]

df_cabin_clean = df_cabin.apply(clean_cabin)

df['Cabin'] = df_cabin_clean

df.drop(columns=['PassengerId', 'Name'], inplace=True)

df = pd.get_dummies(df, columns=['HomePlanet', 'Cabin', 'Destination'])

**Perform Train Test Split**

In [24]:
features = df.drop(columns = ["Transported"])
target = df["Transported"]

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=0)

normalizer = MinMaxScaler()

normalizer.fit(X_train)

X_train_norm_np = normalizer.transform(X_train)
X_test_norm_np = normalizer.transform(X_test)

X_train_norm_df = pd.DataFrame(X_train_norm_np, columns = X_train.columns, index=X_train.index)
X_test_norm_df = pd.DataFrame(X_test_norm_np, columns = X_test.columns, index=X_test.index)

**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [25]:
bagging_reg = BaggingRegressor(DecisionTreeRegressor(max_depth=20),
                               n_estimators=100, # number of models to use
                               max_samples = 1000)

bagging_reg.fit(X_train_norm_df, y_train)

y_pred_test_bag = bagging_reg.predict(X_test_norm_df)

print(f"MAE {mean_absolute_error(y_pred_test_bag, y_test): .2f}")
print(f"RMSE {mean_squared_error(y_pred_test_bag, y_test, squared=False): .2f}")
print(f"R2 score {bagging_reg.score(X_test_norm_df, y_test): .2f}")

MAE  0.28
RMSE  0.38
R2 score  0.43


c:\Users\MJ\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


- Random Forests

In [28]:
forest = RandomForestRegressor(n_estimators=100, max_depth=20)#your code here

forest.fit(X_train_norm_df, y_train)

y_pred_test_rf = forest.predict(X_test_norm_df)

print(f"MAE, {mean_absolute_error(y_pred_test_rf, y_test): .2f}")
print(f"RMSE, {mean_squared_error(y_pred_test_rf, y_test, squared=False): .2f}")
print(f"R2 score, {forest.score(X_test_norm_df, y_test): .2f}")

MAE,  0.27
RMSE,  0.38
R2 score,  0.41


c:\Users\MJ\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


- Gradient Boosting

In [30]:
gb_reg = GradientBoostingRegressor(max_depth=20, n_estimators=100)

gb_reg.fit(X_train_norm_df, y_train)

y_pred_test_gb = gb_reg.predict(X_test_norm_df)

print(f"MAE, {mean_absolute_error(y_pred_test_gb, y_test): .2f}")
print(f"RMSE, {mean_squared_error(y_pred_test_gb, y_test, squared=False): .2f}")
print(f"R2 score, {gb_reg.score(X_test_norm_df, y_test): .2f}")

MAE,  0.26
RMSE,  0.42
R2 score,  0.29


c:\Users\MJ\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


- Adaptive Boosting

In [29]:
ada_reg = AdaBoostRegressor(DecisionTreeRegressor(max_depth=20), n_estimators=100)

ada_reg.fit(X_train_norm_df, y_train)

y_pred_test_ada = ada_reg.predict(X_test_norm_df)

print(f"MAE, {mean_absolute_error(y_pred_test_ada, y_test): .2f}")
print(f"RMSE, {mean_squared_error(y_pred_test_ada, y_test, squared=False): .2f}")
print(f"R2 score, {ada_reg.score(X_test_norm_df, y_test): .2f}")

MAE,  0.25
RMSE,  0.43
R2 score,  0.26


c:\Users\MJ\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Which model is the best and why?

Bagging and pasting has the best scores out of all the ensemble methods that were tested.